<a href="https://colab.research.google.com/github/bahgat-ahmed/Natural-Language-Processing-in-TensorFlow/blob/main/Course_3_Week_2_Lesson_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)
# if your tensorflow version is not 2.x
# tf.enable_paper_execution()
# instaLling tensorflow datasets
!pip install -q tensorflow-datasets

2.3.0


In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQVN2YY/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQVN2YY/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQVN2YY/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  # The values for S and I are tensors, so by calling their NumPy method (.numpy()), I'll actually extract their value.
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

# When training, my labels are expected to be NumPy arrays. So I'll turn the list of labels that I've just created into 
# NumPy arrays with this code instead of being just straight arrays
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [21]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
# the tokenizer will then be fit on the training sentences, not the testing sentences just the training ones 
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
# I then convert my sentences into sequences of numbers, with the number being the value and the word being the 
# key that were taken out of the training sentences when I did fit_on_texts, and that will give me now my list of
# integers per sentence.
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

Do note that the testing sequences are tokenized based on the word index that was learned from the training words. So you may find a lot more OOVs, in the testing sequence than you would have in the training sequence, because there'll be a lot of words that it hasn't encountered. But that's what makes it a good test, because later if you're going to try out a review, you want to be able to do it to see how it will do with words that the tokenizer or the neural network hasn't previously seen. 

In [22]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [23]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '<OOV>') for i in text]) # if it didn't find the index, return <OOV>

print(decode_review(padded[2]))
print(training_sentences[2])

mann photographs the <OOV> rocky mountains in a superb fashion and jimmy stewart and walter brennan give enjoyable performances as they always seem to do br br but come on hollywood a <OOV> telling the people of dawson city <OOV> to <OOV> themselves a <OOV> yes a <OOV> and to <OOV> the law themselves then <OOV> battling it out on the streets for control of the town br br nothing even remotely resembling that happened on the canadian side of the border during the <OOV> gold rush mr mann and company appear to have mistaken dawson city for <OOV> the canadian north for the american wild west br br canadian viewers be prepared for a <OOV> madness type of enjoyable
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then 

The results of the embedding will be a 2D array with the length of the sentence and the embedding dimension for example 16 as its size. So we need to flatten it out in much the same way as we needed to flatten out our images. We then feed that into a dense neural network to do the classification.

In [37]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


Often in natural language processing, a different layer type than a `Flatten()` is used, and this is a global average pooling 1D. The reason for this is the size of the output vector being fed into the dense. Global Average Pooling 1D averages across the vector to flatten it out. Your model summary should look like this, which is simpler and should be a little faster but it will be a little bit less accurate.

>Notice there are 16 outputs instead of 1920 when we used `Flatten()`.

In [38]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


Here's the summary, it all looks good. Again, each of my sentences, 120 characters, my embedding has 16, and out of that the flattened thing we'll have 1,920 values. They get fed into the dense. They get fed into the output layer.

In [39]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4946 - accuracy: 0.7417 - val_loss: 0.3419 - val_accuracy: 0.8512
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2425 - accuracy: 0.9062 - val_loss: 0.3602 - val_accuracy: 0.8420
Epoch 3/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0986 - accuracy: 0.9737 - val_loss: 0.4426 - val_accuracy: 0.8318
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0251 - accuracy: 0.9966 - val_loss: 0.5253 - val_accuracy: 0.8276
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0075 - accuracy: 0.9991 - val_loss: 0.5846 - val_accuracy: 0.8280
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.6393 - val_accuracy: 0.8297
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 9.2911e-04 - accuracy: 1.0000 - val_loss: 0.6842 - val_accuracy: 0.8298
Ep

In [44]:
# the model that used GlobalAveragePooling1D() instead of Flatten()
num_epochs = 10
model2.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5649 - accuracy: 0.7456 - val_loss: 0.3947 - val_accuracy: 0.8416
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.3411 - accuracy: 0.8583 - val_loss: 0.3352 - val_accuracy: 0.8562
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2768 - accuracy: 0.8879 - val_loss: 0.3294 - val_accuracy: 0.8580
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2405 - accuracy: 0.9066 - val_loss: 0.3363 - val_accuracy: 0.8567
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2140 - accuracy: 0.9192 - val_loss: 0.3546 - val_accuracy: 0.8515
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1927 - accuracy: 0.9278 - val_loss: 0.3802 - val_accuracy: 0.8434
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1749 - accuracy: 0.9372 - val_loss: 0.4111 - val_accuracy: 0.8387
Epoch 

So we can see even by epoch seven our accuracy up to a one. Our validation accuracy is still in the low 80s. Pretty good, but this is clear overfitting going on. 

So now let's take a look at what we'll do to view this in the embedding projector. So first of all, I'm going to take the output of my embedding, which was modeled out layer zero, and we can see that there were 10,000 possible words and I had 16 dimensions.

In [45]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


Here is where I'm going to iterate through that array to pull out the values for the 16 dimensions per word and write that as out_V, which is my vectors.tsv. Then the actual word associated with that will be written to out_M, which is my meta.tsv.

So, to the vectors file, we simply write out the value of each of the items in the array of embeddings, i.e, the co-efficient of each dimension on the vector for this word. To the metadata array, we just write out the words. 

In [46]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

If you're working in Colab, this code will download the two files. 

To now render the results, go to the TensorFlow Embedding Projector on projector.tensorflow.org, press the ''Load data'' button on the left. You'll see a dialog asking you to load data from your computer. Use vector.TSV for the first one, and meta.TSV for the second. Once they're loaded, you should see something like this. Click this ''sphereize data'' checkbox on the top left, and you'll see the binary clustering of the data. Experiment by searching for words, or clicking on the blue dots in the chart that represent words. Above all, have some fun with it.

In [47]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
